In [58]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

In [ ]:
path = '/content/drive/MyDrive/DATASET/dataset and all/test_set_features.csv'
df_test = pd.read_csv(path)
df_test

In [60]:
df_test['age_group'] = df_test['age_group'].replace({'18 - 34 Years': 1, '35 - 44 Years': 2, '45 - 54 Years': 3,'55 - 64 Years':4,'65+ Years':5})
df_test['education']  = df_test['education'].replace({'< 12 Years': 1, '12 Years': 2, 'Some College': 3,'College Graduate':4})
df_test['income_poverty'] = df_test['income_poverty'].replace({'<= $75,000, Above Poverty':3,'> $75,000':2,'Below Poverty':1})

In [61]:
cols_to_use  = ['respondent_id', 'xyz_concern', 'xyz_knowledge',
       'behavioral_antiviral_meds', 'behavioral_avoidance',
       'behavioral_face_mask', 'behavioral_wash_hands',
       'behavioral_large_gatherings', 'behavioral_outside_home',
       'behavioral_touch_face', 'doctor_recc_xyz', 'doctor_recc_seasonal',
       'chronic_med_condition', 'child_under_6_months', 'health_worker',
       'health_insurance', 'opinion_xyz_vacc_effective', 'opinion_xyz_risk',
       'opinion_xyz_sick_from_vacc', 'opinion_seas_vacc_effective',
       'opinion_seas_risk', 'opinion_seas_sick_from_vacc', 'age_group',
       'education', 'income_poverty',
       'household_adults', 'household_children',
      ]
df_test1 = df_test[cols_to_use]

In [62]:
df_test2  = df_test1.dropna()

In [63]:
df_test3 = pd.get_dummies(df_test2, drop_first=True)

In [64]:
X_test21 = df_test3.drop(columns=['respondent_id'])

In [65]:
train = '/content/drive/MyDrive/DATASET/dataset and all/training_set_features.csv'
df  = pd.read_csv(train)

In [66]:
train_out = '/content/drive/MyDrive/DATASET/dataset and all/training_set_labels.csv'
df1 = pd.read_csv(train_out)

In [67]:
df2 = pd.concat([df, df1], axis=1)

In [68]:
df2['age_group'] = df2['age_group'].replace({'18 - 34 Years': 1, '35 - 44 Years': 2, '45 - 54 Years': 3,'55 - 64 Years':4,'65+ Years':5})
df2['education']  = df2['education'].replace({'< 12 Years': 1, '12 Years': 2, 'Some College': 3,'College Graduate':4})
df2['income_poverty'] = df2['income_poverty'].replace({'<= $75,000, Above Poverty':3,'> $75,000':2,'Below Poverty':1})

In [ ]:
cols_to_use  = ['respondent_id', 'xyz_concern', 'xyz_knowledge',
       'behavioral_antiviral_meds', 'behavioral_avoidance',
       'behavioral_face_mask', 'behavioral_wash_hands',
       'behavioral_large_gatherings', 'behavioral_outside_home',
       'behavioral_touch_face', 'doctor_recc_xyz', 'doctor_recc_seasonal',
       'chronic_med_condition', 'child_under_6_months', 'health_worker',
       'health_insurance', 'opinion_xyz_vacc_effective', 'opinion_xyz_risk',
       'opinion_xyz_sick_from_vacc', 'opinion_seas_vacc_effective',
       'opinion_seas_risk', 'opinion_seas_sick_from_vacc', 'age_group',
       'education', 'income_poverty',
       'household_adults', 'household_children',
      'xyz_vaccine',
       'seasonal_vaccine']
df3 = df2[cols_to_use]
df3.isnull().sum()

In [70]:
df4  = df3.dropna()

In [71]:
df5 = pd.get_dummies(df4, drop_first=True)

In [72]:
X = df5.drop(columns=['respondent_id', 'xyz_vaccine', 'seasonal_vaccine'])
y = df5[['xyz_vaccine', 'seasonal_vaccine']]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=7)
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
X_train_scaled=scaler.fit_transform(X_train)
X_train_scaled

In [74]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(X_train, y_train)
reg.score(X_test, y_test) # 0.38
reg.score(X_train, y_train) #0.36

0.36433669455427437

In [75]:
# SO I USED LOGISTIC REGRESSION
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression().fit(X_train, y_train['xyz_vaccine'])
# log_reg.score(X_test, y_test['xyz_vaccine'])  0.82 so decent score

xyz_predictions = log_reg.predict_proba(X_test)[:,1]
print(xyz_predictions)

[0.1732344  0.9138037  0.5282203  ... 0.5560406  0.70040603 0.10956669]


In [76]:
test_xyz = log_reg.predict_proba(X_test21)[:,1]
print(test_xyz)

[0.1543032  0.02682978 0.55983738 ... 0.08615774 0.05248093 0.20610355]


In [77]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression().fit(X_train, y_train['seasonal_vaccine'])


seasonal_predictions = log_reg.predict_proba(X_test)[:,1]
print(seasonal_predictions)

[0.20065314 0.97638923 0.74309289 ... 0.77319839 0.93469155 0.1679102 ]


In [78]:
test_seasonal = log_reg.predict_proba(X_test21)[:,1]
print(test_seasonal)

[0.29564302 0.0322604  0.88356903 ... 0.19284098 0.17331116 0.27874065]


In [79]:
roc_auc_xyz = roc_auc_score(y_test['xyz_vaccine'], xyz_predictions)
roc_auc_seasonal = roc_auc_score(y_test['seasonal_vaccine'], seasonal_predictions)
print(roc_auc_xyz)
print(roc_auc_seasonal)

0.8638369740833675
0.873958516803012


In [84]:
final_submit  = pd.DataFrame({'respondent_id': df_test2['respondent_id'], 'xyz_vaccine': test_xyz, 'seasonal_vaccine': test_seasonal})
final_submit.to_csv('final_submit.csv', index=False)